# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [5]:
#from google.colab import drive

In [6]:
#drive.mount('/content/drive/')

#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [7]:
#project_path = "/content/drive/My Drive/Colab Notebooks/NLP_LSTM/"

### Loading the Glove Embeddings

In [8]:
from zipfile import ZipFile
with ZipFile('glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [9]:
import pandas as pd

train_bodies = pd.read_csv('train_bodies.csv')
train_stances = pd.read_csv('train_stances.csv')

In [10]:
dataset = pd.merge(train_bodies[['Body ID','articleBody']],train_stances [['Body ID','Headline','Stance']],left_on = 'Body ID', right_on = 'Body ID', how = 'inner')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [11]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [12]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\20012\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [14]:
from keras.preprocessing.text import Tokenizer 

Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [15]:
t= Tokenizer(num_words = MAX_NB_WORDS,filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}\n"~')

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [16]:
t.fit_on_texts(dataset['articleBody'])
t.fit_on_texts(dataset['Headline'])

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



In [17]:
vocab_size = len(t.word_index)+1
print(vocab_size)

27879


### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [18]:
from nltk import tokenize

texts = []
articles = []

for idx in range ( dataset [ 'articleBody'].shape[0]):
  text = dataset['articleBody'][idx]
  texts.append(text)
  sentences = tokenize.sent_tokenize(text)
  articles.append(sentences)

## Check 2:

first element of texts and articles should be as given below. 

In [19]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [20]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [21]:
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

data = np.zeros((len(texts),MAX_SENTS,MAX_SENT_LENGTH),dtype = 'int32')
for i,sentences in enumerate(articles):
    for j,sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
        for _,word in enumerate(wordTokens):
            try:
                if k < MAX_SENT_LENGTH and t.word_index[word] < MAX_NB_WORDS:
                    data[i,j,k] = t.word_index[word]
                    k=k+1
            except:
                #print(word)
                pass
        

### Check 3:

Accessing first element in data should give something like given below.

In [22]:
data[0, :, :]

array([[    3,   481,   427,  7211,    81,     3,  3733,   331,     5,
         3891,   350,     4,  1431,  2958,     1,    89,    12,   464,
            0,     0],
       [  758,    95,  1047,     3,  2679,  1752,     7,   189,     3,
         1217,  1075,  2030,   700,   159,     1,  3032,   448,     1,
          555,   235],
       [   89,  1067,  4115,  2349,    12,     3,  1092,  3306,    19,
            1,    89,     2,  1793,     1,   521,  2009,    15,     9,
            3,  3111],
       [  181,  3640,   972,   200,  2556,    44,  6775,  1722,  1252,
            5, 13319, 17939,     1,   778,    31,   740,  3990,    67,
           85,     0],
       [ 2349,    12,  1557,    38,  1094,   351,   775,     2,   367,
          260,  1770,     5,  4450,    70,   494,     0,     0,     0,
            0,     0],
       [    1,   700,   189,    19,     1,   427,    32,     3,  7417,
            4,  2159,  1252,     6,     3,  5270,     4,  1217,  1252,
           12,  3363],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [23]:
text_heading = []
articles_heading = []

for idx in range(dataset['Headline'].shape[0]):
    text = dataset['Headline'][idx]
    text_heading.append(text)
    sentences = tokenize.sent_tokenize(text)
    articles_heading.append(sentences)

data_heading = np.zeros((len(texts),MAX_SENTS_HEADING,MAX_SENT_LENGTH),dtype= 'int32')

for i,sentences in enumerate(articles_heading):
    for j,sent in enumerate(sentences):
        if j < MAX_SENTS_HEADING:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _,word in enumerate(wordTokens):
                try:
                    if k < MAX_SENT_LENGTH and t.word_index[word] < MAX_NB_WORDS:
                        data_heading[ i , j , k] = t.word_index[word]
                        k = k + 1
                except :
                    pass
                    
      

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [24]:
labels = dataset['Stance']
targets = pd.Series(labels)
one_hot = pd.get_dummies(targets,sparse = True)
one_hot_labels = np.asarray(one_hot)
labels = one_hot_labels

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [25]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [26]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [27]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [28]:
# Validation data
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [29]:
### for train and validation samples 
x_train = data[:-nb_validation_samples]
x_heading_train = data_heading[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
x_heading_val = data_heading[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [30]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39978, 20, 20)
(39978, 4)
(9994, 20, 20)
(9994, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [31]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [32]:
from keras.models import Sequential
from keras.layers import Dense,LSTM, TimeDistributed, Activation
from keras.layers import Flatten, Permute, merge, Input
from keras.layers import Embedding
from keras.models import Model
from keras.layers import Input,Dense,multiply,concatenate,Dropout

### Model

In [33]:
MAX_SENT_LENGTH = 20
MAX_SENTS = 20
hidden_size = 100

from keras.layers import GRU, Bidirectional

sentence_input = Input(shape=(MAX_SENT_LENGTH,),dtype='int32')
print(sentence_input)
embedded_sequences = Embedding(output_dim = hidden_size, input_dim = vocab_size, input_length = (MAX_SENT_LENGTH,),)(sentence_input)

l_LSTM = Bidirectional(LSTM(100,return_sequences = True))(embedded_sequences)
l_dense = TimeDistributed(Dense(100))(l_LSTM)
l_dense = Flatten()(l_dense)
sentEncoder = Model(sentence_input,l_dense)

body_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH,),dtype = 'int32')
print(body_input)
body_encoder = TimeDistributed(sentEncoder)(body_input)
print(body_encoder)
l_LSTM_sent = Bidirectional(LSTM(100,return_sequences=True))(body_encoder)
l_dense_sent = TimeDistributed(Dense(100))(l_LSTM_sent)
l_dense_sent = Flatten()(l_dense_sent)

heading_input = Input(shape = (MAX_SENTS_HEADING, MAX_SENT_LENGTH),dtype = 'int32')
heading_embedded_sequences = Embedding(output_dim=hidden_size, input_dim=vocab_size, \
                                       input_length = (MAX_SENTS_HEADING,MAX_SENT_LENGTH,), \
                                      weights = [embedding_matrix])(heading_input)
h_dense = Dense(100,activation='relu')(heading_embedded_sequences)
h_flatten = Flatten()(h_dense)
article_output = concatenate([l_dense_sent,h_flatten],name = 'concatenate_heading')

news_vestor = Dense(100,activation = 'relu')(article_output)
preds = Dense(4,activation = 'softmax')(news_vestor)
model = Model([body_input,heading_input],[preds])

                   

W1207 13:58:56.984269  2036 deprecation_wrapper.py:119] From C:\Users\20012\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1207 13:58:57.063221  2036 deprecation_wrapper.py:119] From C:\Users\20012\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1207 13:58:57.085209  2036 deprecation_wrapper.py:119] From C:\Users\20012\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Tensor("input_1:0", shape=(?, 20), dtype=int32)
Tensor("input_2:0", shape=(?, 20, 20), dtype=int32)
Tensor("time_distributed_2/Reshape_1:0", shape=(?, 20, 2000), dtype=float32)


W1207 13:58:58.738262  2036 deprecation_wrapper.py:119] From C:\Users\20012\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1207 13:58:58.739261  2036 deprecation_wrapper.py:119] From C:\Users\20012\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



### Compile and fit the model

In [34]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics = ['acc'])
print("model fitting = Hierarchical LSTM network")
model.fit([x_train,x_heading_train],[y_train],validation_data = ([x_val,x_heading_val],[y_val]),epochs = 10 , batch_size=50)

W1207 13:58:59.263803  2036 deprecation_wrapper.py:119] From C:\Users\20012\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1207 13:58:59.464965  2036 deprecation.py:323] From C:\Users\20012\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


model fitting = Hierarchical LSTM network
Train on 39978 samples, validate on 9994 samples
Epoch 1/10
39978/39978 [==============================] - 1304s 33ms/step - loss: 0.5810 - acc: 0.7898 - val_loss: 0.4254 - val_acc: 0.8393
Epoch 2/10
39978/39978 [==============================] - 1264s 32ms/step - loss: 0.3352 - acc: 0.8743 - val_loss: 0.3160 - val_acc: 0.8793
Epoch 3/10
39978/39978 [==============================] - 1273s 32ms/step - loss: 0.2381 - acc: 0.9092 - val_loss: 0.2893 - val_acc: 0.8905
Epoch 4/10
39978/39978 [==============================] - 1269s 32ms/step - loss: 0.1797 - acc: 0.9326 - val_loss: 0.2446 - val_acc: 0.9193
Epoch 5/10
39978/39978 [==============================] - 1269s 32ms/step - loss: 0.1426 - acc: 0.9487 - val_loss: 0.2671 - val_acc: 0.9143
Epoch 6/10
39978/39978 [==============================] - 1258s 31ms/step - loss: 0.1160 - acc: 0.9585 - val_loss: 0.3399 - val_acc: 0.8936
Epoch 7/10
39978/39978 [==============================] - 1307s 33ms/

#### We could see the accuracy of 97%, with validation accuracy of 92%. This could be improved by increasing the epoch or batch size.

## Build the same model with attention layers included for better performance (Optional)

In [50]:
EMBEDDING_DIM = 100
word_index = t.word_index

# building Hierachical Attention network
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True,
                            mask_zero=True)

## Fit the model and report the accuracy score for the model with attention layer (Optional)

In [55]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)), name='W')
        self.b = K.variable(self.init((self.attention_dim, )), name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)), name='u')
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = Embedding(output_dim = hidden_size, input_dim = vocab_size, input_length = (MAX_SENT_LENGTH,),)(sentence_input)

#embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_lstm)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_att_sent = AttLayer(100)(l_lstm_sent)
preds = Dense(4, activation='softmax')(l_att_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])

print("model fitting - Hierachical attention network")
model.fit(x_train, y_train, validation_data=(x_val, y_val),nb_epoch=10, batch_size=50)

model fitting - Hierachical attention network


C:\Users\20012\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:61: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 39978 samples, validate on 9994 samples
Epoch 1/10
39978/39978 [==============================] - 969s 24ms/step - loss: 0.6739 - acc: 0.7539 - val_loss: 0.6012 - val_acc: 0.7690
Epoch 2/10
39978/39978 [==============================] - 973s 24ms/step - loss: 0.5594 - acc: 0.7947 - val_loss: 0.5433 - val_acc: 0.7996
Epoch 3/10
39978/39978 [==============================] - 972s 24ms/step - loss: 0.5278 - acc: 0.8086 - val_loss: 0.5453 - val_acc: 0.8020
Epoch 4/10
39978/39978 [==============================] - 982s 25ms/step - loss: 0.5132 - acc: 0.8138 - val_loss: 0.5334 - val_acc: 0.8049
Epoch 5/10
39978/39978 [==============================] - 973s 24ms/step - loss: 0.5022 - acc: 0.8164 - val_loss: 0.5361 - val_acc: 0.8101
Epoch 6/10
39978/39978 [==============================] - 986s 25ms/step - loss: 0.4970 - acc: 0.8171 - val_loss: 0.5159 - val_acc: 0.8105
Epoch 7/10
39978/39978 [==============================] - 978s 24ms/step - loss: 0.4914 - acc: 0.8194 - val_loss: 0.5